Step 1: Install yfinance

In [ ]:
!pip install yfinance

Step 2: Import Libraries

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

Step 3: Get Stock Data

In [ ]:
ticker = "RELIANCE.NS"  # Change as needed
start_date = "2020-01-01" # Adjust as needed
end_date = "2025-03-12"   # Adjust as needed

data = yf.download(ticker, start=start_date, end=end_date)
# print(data.head()) # Uncomment to see data

Step 4: Calculate Returns and Volatility

In [ ]:
data['Daily_Return'] = data['Close'].pct_change()
data = data.dropna()

volatility = data['Daily_Return'].std() * np.sqrt(252)
drift = data['Daily_Return'].mean() * 252

print(f"Annual Volatility: {volatility:.4f}")
print(f"Annual Drift: {drift:.4f}")

Step 5: Monte Carlo Simulation Function

In [ ]:
def monte_carlo_simulation(last_price, days, num_simulations, drift, volatility):
    dt = 1/252
    price_paths = np.zeros((num_simulations, days + 1))
    price_paths[:, 0] = last_price

    for i in range(num_simulations):
        for j in range(1, days + 1):
            Z = np.random.normal(0, 1)
            price_paths[i, j] = price_paths[i, j-1] * np.exp((drift - 0.5 * volatility**2) * dt + volatility * np.sqrt(dt) * Z)

    return price_paths

Step 6: Run the Simulation

In [ ]:
last_known_price = data['Close'].iloc[-1]
prediction_days = 30
num_simulations = 1000

simulated_paths = monte_carlo_simulation(last_known_price, prediction_days, num_simulations, drift, volatility)

Step 7: Visualize Results

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(num_simulations):
    plt.plot(simulated_paths[i, :], linewidth=0.5, alpha=0.3)

plt.title(f'Monte Carlo Simulation for {ticker} - {num_simulations} Paths')
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.grid(True)
plt.show()

ending_prices = simulated_paths[:, -1]
plt.figure(figsize=(8, 5))
plt.hist(ending_prices, bins=50, density=True, alpha=0.6, color='skyblue')
plt.title(f'Distribution of Predicted Prices after {prediction_days} Days')
plt.xlabel('Stock Price')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

Step 8: Analyze Results

In [ ]:
average_predicted_price = np.mean(ending_prices)
print(f"Average Predicted Price after {prediction_days} days: {average_predicted_price:.2f}")

confidence_interval = np.percentile(ending_prices, [2.5, 97.5])
print(f"95% Confidence Interval for Predicted Price: {confidence_interval}")